In [1]:
%matplotlib inline

In [2]:
# How to get feature names?????
from sklearn.datasets import load_boston
other_dataset = load_boston()
df = pd.DataFrame(other_dataset.data, columns=other_dataset.feature_names)
df['target'] = other_dataset.target
feature_names = other_dataset.feature_names
print(other_dataset.DESCR)

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

In [3]:
(train_data, train_targets), (test_data, test_targets) = ks.datasets.boston_housing.load_data() 

In [4]:
(train_data, train_targets), (test_data, test_targets) = ks.datasets.boston_housing.load_data()

## Normalize the data, by column
mean= train_data.mean(axis=0)
print("Subtracting the mean:", mean)
train_data -= mean
std = train_data.std(axis=0)
print("Dividing by the std:", std)
train_data /= std

# Apply normalization to the test set as well
test_data -= mean
test_data /= std


Subtracting the mean: [  3.74511057e+00   1.14801980e+01   1.11044307e+01   6.18811881e-02
   5.57355941e-01   6.26708168e+00   6.90106436e+01   3.74027079e+00
   9.44059406e+00   4.05898515e+02   1.84759901e+01   3.54783168e+02
   1.27408168e+01]
Dividing by the std: [  9.22929073e+00   2.37382770e+01   6.80287253e+00   2.40939633e-01
   1.17147847e-01   7.08908627e-01   2.79060634e+01   2.02770050e+00
   8.68758849e+00   1.66168506e+02   2.19765689e+00   9.39946015e+01
   7.24556085e+00]


In [5]:
lengths = list()
max_indices = list()
for el in train_data[0:5]:
    lengths.append(len(el))
    max_indices.append(max(el))
print("Training data: ",train_data.shape[0]," examples, with",train_data.shape[1], "features")
print("Training output: ",train_targets.shape)
print("")
print("Test data: ",test_data.shape[0]," examples, with",test_data.shape[1], "features")
print("Test output: ",train_targets.shape)

print("Test data shape: ",test_data.shape)
print("Train data 0: {}".format(train_data[0]))
print("Type: {}".format(type(train_data[0])))
print("Maximum length: {}".format(max(lengths)))
print("Average length: {}".format(np.mean(lengths)))
print("Largest index: {}".format(max(max_indices)))

Training data:  404  examples, with 13 features
Training output:  (404,)

Test data:  102  examples, with 13 features
Test output:  (404,)
Test data shape:  (102, 13)
Train data 0: [-0.27224633 -0.48361547 -0.43576161 -0.25683275 -0.1652266  -0.1764426
  0.81306188  0.1166983  -0.62624905 -0.59517003  1.14850044  0.44807713
  0.8252202 ]
Type: <class 'numpy.ndarray'>
Maximum length: 13
Average length: 13.0
Largest index: 2.991784193632328


In [6]:
this_df = pd.DataFrame(train_data, )
this_df.columns = feature_names
#this_df.head()

In [7]:
#this_df.describe()

## PAIR PLOT

In [8]:
#sns.pairplot(this_df)

In [17]:
ks.models
ks.layers

def build_model():
    model = ks.models.Sequential()
    model.add(ks.layers.Dense(64, activation='relu', input_shape = (train_data.shape[1],)))
    model.add(ks.layers.Dense(64, activation='relu'))
    model.add(ks.layers.Dense(1))
    
    optimizer = 'rmsprop'
    loss = 'mse' # Mean squared error
    metrics = ['mae']
    model.compile(optimizer=optimizer, loss=loss, metrics = metrics)
    
    return model


In [18]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 64)                896       
_________________________________________________________________
dense_29 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 65        
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________


In [28]:
k = 4
print("k:", k)
num_val_samples = len(train_data) // k
print("num_val_samples",num_val_samples)
num_epochs = 100
all_scores = list()
all_history = list()
for i in range(k):
    print("Fold:",i)
    val_data = train_data[i*num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i*num_val_samples: (i + 1) * num_val_samples]

    left_data = train_data[:i * num_val_samples]
    right_data = train_data[(i+1)  * num_val_samples:]
    partial_train_data = np.concatenate([left_data, right_data], axis=0)
    
    left_targets = train_targets[:i * num_val_samples]
    right_targets = train_targets[(i+1)  * num_val_samples:]
    partial_train_targets = np.concatenate([left_targets, right_targets], axis=0)
    
    print(val_data.shape, val_targets.shape)
    print(partial_train_data.shape, partial_train_targets.shape)
    
    model = build_model()
    this_history = model.fit(
        partial_train_data, 
        partial_train_targets,
        epochs = num_epochs,
        batch_size = 1,
        verbose = False)
    
    #print(val_data)
    #print(val_targets)
    
    #print(type(val_data))
    #print(type(val_targets))
    
    #val_mse, val_mae = 
    result = model.evaluate(val_data, val_targets, verbose = 0)
    all_history.append(this_history)
    all_scores.append(result)
print("Done")
        

k: 4
num_val_samples 101
Fold: 0
(101, 13) (101,)
(303, 13) (303,)
Fold: 1
(101, 13) (101,)
(303, 13) (303,)
Fold: 2
(101, 13) (101,)
(303, 13) (303,)
Fold: 3
(101, 13) (101,)
(303, 13) (303,)
Done


In [29]:
# All scores
print(all_scores)
mean_scores = np.mean(all_scores,axis = 0)
print("Mean Squared Error:",mean_scores[0],"Absolute Error:",mean_scores[1])

[[8.4497931546503953, 2.1677483497279706], [12.430517952040871, 2.5807215081583155], [14.788055384513175, 2.5327313949566075], [12.908984082760197, 2.5581954906482509]]
Mean Squared Error: 12.1443376435 Absolute Error: 2.45984918587
